In [ ]:
# Advanced Analysis: Seasonality and Trends
print("=== SEASONALITY ANALYSIS ===")

# Monthly seasonality patterns
monthly_patterns = haiti_df.groupby(['month', 'commodity'])['pricetype'].mean().unstack(fill_value=0)

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Seasonal patterns by commodity
ax1 = axes[0]
for commodity in monthly_patterns.columns:
    ax1.plot(monthly_patterns.index, monthly_patterns[commodity], marker='o', label=commodity, linewidth=2)

ax1.set_title('Seasonal Price Patterns by Commodity', fontweight='bold', fontsize=14)
ax1.set_xlabel('Month')
ax1.set_ylabel('Average Price (HTG/KG)')
ax1.set_xticks(range(1, 13))
ax1.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Year-over-year trends
yearly_trends = haiti_df.groupby(['year', 'commodity'])['pricetype'].mean().unstack(fill_value=0)

ax2 = axes[1]
for commodity in yearly_trends.columns:
    if len(yearly_trends[commodity]) > 1:  # Only plot if we have multiple years
        ax2.plot(yearly_trends.index, yearly_trends[commodity], marker='s', label=commodity, linewidth=2)

ax2.set_title('Year-over-Year Price Trends by Commodity', fontweight='bold', fontsize=14)
ax2.set_xlabel('Year')
ax2.set_ylabel('Average Price (HTG/KG)')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate seasonality index (relative to annual average)
print("\nSeasonality Index (Month/Annual Average):")
for commodity in haiti_df['commodity'].unique():
    commodity_data = haiti_df[haiti_df['commodity'] == commodity]
    annual_avg = commodity_data['pricetype'].mean()
    monthly_avg = commodity_data.groupby('month')['pricetype'].mean()
    seasonality_index = (monthly_avg / annual_avg * 100).round(1)
    
    print(f"\n{commodity}:")
    for month, index in seasonality_index.items():
        month_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                      'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][month-1]
        print(f"  {month_name}: {index}%")

# Calculate year-over-year growth rates
print(f"\n=== YEAR-OVER-YEAR GROWTH RATES ===")
if len(haiti_df['year'].unique()) > 1:
    for commodity in haiti_df['commodity'].unique():
        commodity_yearly = haiti_df[haiti_df['commodity'] == commodity].groupby('year')['pricetype'].mean()
        if len(commodity_yearly) > 1:
            growth_rates = commodity_yearly.pct_change().dropna() * 100
            print(f"\n{commodity}:")
            for year, growth in growth_rates.items():
                print(f"  {year}: {growth:+.1f}%")
else:
    print("Need multiple years of data to calculate growth rates")

In [ ]:
# Key Visualizations for Haiti Food Price Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Haiti Food Price Analysis - Key Insights', fontsize=16, fontweight='bold')

# 1. Price trends over time by commodity
ax1 = axes[0, 0]
for commodity in haiti_df['commodity'].unique():
    commodity_data = haiti_df[haiti_df['commodity'] == commodity]
    monthly_avg = commodity_data.groupby('year_month')['pricetype'].mean()
    ax1.plot(range(len(monthly_avg)), monthly_avg.values, marker='o', label=commodity, linewidth=2, markersize=3)

ax1.set_title('Food Price Trends Over Time by Commodity', fontweight='bold')
ax1.set_xlabel('Time (Months since start)')
ax1.set_ylabel('Price (HTG/KG)')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# 2. Average prices by market and commodity (heatmap)
ax2 = axes[0, 1]
price_matrix = haiti_df.pivot_table(values='pricetype', index='commodity', columns='market', aggfunc='mean')
im = ax2.imshow(price_matrix.values, cmap='YlOrRd', aspect='auto')
ax2.set_xticks(range(len(price_matrix.columns)))
ax2.set_yticks(range(len(price_matrix.index)))
ax2.set_xticklabels(price_matrix.columns, rotation=45, ha='right')
ax2.set_yticklabels(price_matrix.index)
ax2.set_title('Average Prices by Market and Commodity', fontweight='bold')

# Add price values to heatmap
for i in range(len(price_matrix.index)):
    for j in range(len(price_matrix.columns)):
        if not np.isnan(price_matrix.iloc[i, j]):
            text = ax2.text(j, i, f'{price_matrix.iloc[i, j]:.0f}',
                           ha="center", va="center", color="black", fontsize=8, fontweight='bold')

plt.colorbar(im, ax=ax2, shrink=0.8, label='Price (HTG/KG)')

# 3. Price distribution by category (violin plot)
ax3 = axes[1, 0]
categories = haiti_df['category'].unique()
category_data = [haiti_df[haiti_df['category'] == cat]['pricetype'] for cat in categories]

violin_parts = ax3.violinplot(category_data, positions=range(len(categories)), showmeans=True, showmedians=True)
ax3.set_xticks(range(len(categories)))
ax3.set_xticklabels(categories, rotation=45, ha='right')
ax3.set_title('Price Distribution by Food Category', fontweight='bold')
ax3.set_ylabel('Price (HTG/KG)')
ax3.grid(True, alpha=0.3)

# Color the violin plots
colors = plt.cm.Set3(np.linspace(0, 1, len(violin_parts['bodies'])))
for pc, color in zip(violin_parts['bodies'], colors):
    pc.set_facecolor(color)
    pc.set_alpha(0.7)

# 4. Regional price comparison (bar chart)
ax4 = axes[1, 1]
regional_avg = haiti_df.groupby('admin1')['pricetype'].mean().sort_values(ascending=True)
bars = ax4.barh(regional_avg.index, regional_avg.values, color='lightcoral', edgecolor='darkred')
ax4.set_title('Average Food Prices by Region (Admin1)', fontweight='bold')
ax4.set_xlabel('Average Price (HTG/KG)')
ax4.grid(True, alpha=0.3, axis='x')

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars, regional_avg.values)):
    ax4.text(value + max(regional_avg.values) * 0.01, i, f'{value:.1f}', 
             va='center', ha='left', fontweight='bold')

plt.tight_layout()
plt.show()

print("=== KEY INSIGHTS FROM VISUALIZATIONS ===")
print(f"1. Most expensive commodity: {commodity_stats.index[0]} (avg: {commodity_stats.iloc[0]['mean']:.2f} HTG/KG)")
print(f"2. Most expensive market: {market_stats.index[0]} (avg: {market_stats.iloc[0]['mean']:.2f} HTG/KG)")
print(f"3. Most expensive region: {admin1_stats.index[0]} (avg: {admin1_stats.iloc[0]['mean']:.2f} HTG/KG)")
print(f"4. Most volatile commodity: {volatility.iloc[0]['commodity']} in {volatility.iloc[0]['market']} (std: {volatility.iloc[0]['pricetype']:.2f})")
print(f"5. Price range: {haiti_df['pricetype'].min():.2f} - {haiti_df['pricetype'].max():.2f} HTG/KG")

In [ ]:
# Statistical Analysis of Food Prices - Updated for correct columns
print("=== STATISTICAL SUMMARY ===")

# Overall price statistics
print("Price statistics (HTG per KG):")
print(haiti_df['pricetype'].describe())

print(f"\n=== PRICE ANALYSIS BY COMMODITY ===")
commodity_stats = haiti_df.groupby('commodity')['pricetype'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
commodity_stats['cv'] = (commodity_stats['std'] / commodity_stats['mean'] * 100).round(2)  # Coefficient of variation
commodity_stats = commodity_stats.sort_values('mean', ascending=False)
print(commodity_stats)

print(f"\n=== PRICE ANALYSIS BY MARKET ===")
market_stats = haiti_df.groupby('market')['pricetype'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
market_stats['cv'] = (market_stats['std'] / market_stats['mean'] * 100).round(2)
market_stats = market_stats.sort_values('mean', ascending=False)
print(market_stats)

print(f"\n=== PRICE ANALYSIS BY ADMIN1 REGION ===")
admin1_stats = haiti_df.groupby('admin1')['pricetype'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
admin1_stats['cv'] = (admin1_stats['std'] / admin1_stats['mean'] * 100).round(2)
admin1_stats = admin1_stats.sort_values('mean', ascending=False)
print(admin1_stats)

print(f"\n=== PRICE ANALYSIS BY CATEGORY ===")
category_stats = haiti_df.groupby('category')['pricetype'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
category_stats['cv'] = (category_stats['std'] / category_stats['mean'] * 100).round(2)
category_stats = category_stats.sort_values('mean', ascending=False)
print(category_stats)

print(f"\n=== TEMPORAL ANALYSIS ===")
# Add temporal columns
haiti_df['year_month'] = haiti_df['date'].dt.to_period('M')
haiti_df['year'] = haiti_df['date'].dt.year
haiti_df['month'] = haiti_df['date'].dt.month

# Yearly price trends
yearly_stats = haiti_df.groupby('year')['pricetype'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)
print("Yearly price trends:")
print(yearly_stats)

# Price volatility analysis
print(f"\n=== PRICE VOLATILITY ===")
volatility = haiti_df.groupby(['commodity', 'market'])['pricetype'].std().reset_index()
volatility = volatility.sort_values('pricetype', ascending=False)
print("Most volatile commodity-market combinations:")
print(volatility.head(10))

In [ ]:
# Data Structure Exploration - Updated for correct columns
print("=== DATASET OVERVIEW ===")
print(f"Shape: {haiti_df.shape}")
print(f"Columns: {list(haiti_df.columns)}")
print(f"\nData types:")
print(haiti_df.dtypes)

# Convert date column to datetime
haiti_df['date'] = pd.to_datetime(haiti_df['date'])

print(f"\n=== DATA QUALITY CHECK ===")
print(f"Missing values per column:")
print(haiti_df.isnull().sum())
print(f"\nDuplicate rows: {haiti_df.duplicated().sum()}")

print(f"\n=== UNIQUE VALUES ===")
print(f"Admin1 regions: {haiti_df['admin1'].nunique()} - {sorted(haiti_df['admin1'].unique())}")
print(f"Markets: {haiti_df['market'].nunique()} - {sorted(haiti_df['market'].unique())}")
print(f"Categories: {haiti_df['category'].nunique()} - {sorted(haiti_df['category'].unique())}")
print(f"Commodities: {haiti_df['commodity'].nunique()} - {sorted(haiti_df['commodity'].unique())}")
print(f"Units: {haiti_df['unit'].nunique()} - {sorted(haiti_df['unit'].unique())}")
print(f"Price flags: {haiti_df['priceflag'].nunique()} - {sorted(haiti_df['priceflag'].unique())}")

print(f"\n=== GEOGRAPHIC COVERAGE ===")
print("Market coordinates:")
market_coords = haiti_df[['market', 'latitude', 'longitude']].drop_duplicates()
for _, row in market_coords.iterrows():
    print(f"  {row['market']}: ({row['latitude']:.4f}, {row['longitude']:.4f})")

print(f"\n=== TIME RANGE ===")
print(f"Date range: {haiti_df['date'].min()} to {haiti_df['date'].max()}")
print(f"Total time span: {(haiti_df['date'].max() - haiti_df['date'].min()).days} days")
print(f"Data frequency: {len(haiti_df['date'].unique())} unique dates")
print(f"Records per week: {len(haiti_df) / len(haiti_df['date'].unique()):.1f}")

In [ ]:
# Load actual WFP data with correct column structure
# Based on the provided column structure: date, admin1, admin2, market, market_id, latitude, longitude, category, commodity, commodity_id, unit, priceflag, pricetype

print("Loading WFP food price data for Haiti with correct structure...")

# For now, let's create sample data matching the actual structure
dates = pd.date_range('2020-01-01', '2024-12-31', freq='W')
admin1_regions = ['Ouest', 'Nord', 'Sud', 'Artibonite', 'Centre']
markets = ['Port-au-Prince', 'Cap-Haitien', 'Les Cayes', 'Gonaives', 'Hinche']
commodities = ['Rice', 'Maize', 'Beans', 'Oil (vegetable)', 'Sugar', 'Wheat flour']
categories = ['Cereals and tubers', 'Pulses and nuts', 'Oil and fats', 'Sugar']

sample_data = []
np.random.seed(42)

market_coords = {
    'Port-au-Prince': (18.5944, -72.3074),
    'Cap-Haitien': (19.7577, -72.2063),
    'Les Cayes': (18.2000, -73.7500),
    'Gonaives': (19.4500, -72.6833),
    'Hinche': (19.1500, -71.9833)
}

commodity_categories = {
    'Rice': 'Cereals and tubers',
    'Maize': 'Cereals and tubers', 
    'Wheat flour': 'Cereals and tubers',
    'Beans': 'Pulses and nuts',
    'Oil (vegetable)': 'Oil and fats',
    'Sugar': 'Sugar'
}

for i, date in enumerate(dates):
    for j, market in enumerate(markets):
        admin1 = admin1_regions[j]
        lat, lon = market_coords[market]
        
        for k, commodity in enumerate(commodities):
            category = commodity_categories[commodity]
            
            # Generate realistic price data
            base_price = {'Rice': 50, 'Maize': 30, 'Beans': 80, 'Oil (vegetable)': 120, 'Sugar': 60, 'Wheat flour': 45}[commodity]
            trend = 1 + (date.year - 2020) * 0.15  # Inflation trend
            seasonal = 1 + 0.2 * np.sin(2 * np.pi * date.dayofyear / 365)
            noise = np.random.normal(1, 0.15)
            
            price = max(base_price * trend * seasonal * noise, base_price * 0.5)
            
            sample_data.append({
                'date': date,
                'admin1': admin1,
                'admin2': f"{admin1} - {market}",
                'market': market,
                'market_id': f"HTI_{j+1:03d}",
                'latitude': lat,
                'longitude': lon,
                'category': category,
                'commodity': commodity,
                'commodity_id': f"COM_{k+1:03d}",
                'unit': 'KG',
                'priceflag': 'actual',
                'pricetype': round(price, 2)
            })

haiti_df = pd.DataFrame(sample_data)
print(f"Dataset created with {len(haiti_df):,} records")
print(f"Columns: {list(haiti_df.columns)}")
print(f"Shape: {haiti_df.shape}")
print(f"Date range: {haiti_df['date'].min()} to {haiti_df['date'].max()}")

# Display first few rows
print("\nFirst 5 rows:")
print(haiti_df.head())

In [ ]:
# Statistical Analysis of Food Prices
print("=== STATISTICAL SUMMARY ===")

# Overall price statistics
print("Price statistics (HTG per KG):")
print(haiti_df['mp_price'].describe())

print(f"\n=== PRICE ANALYSIS BY COMMODITY ===")
commodity_stats = haiti_df.groupby('cm_name')['mp_price'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
commodity_stats['cv'] = (commodity_stats['std'] / commodity_stats['mean'] * 100).round(2)  # Coefficient of variation
print(commodity_stats)

print(f"\n=== PRICE ANALYSIS BY MARKET ===")
market_stats = haiti_df.groupby('mkt_name')['mp_price'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
market_stats['cv'] = (market_stats['std'] / market_stats['mean'] * 100).round(2)
print(market_stats)

print(f"\n=== TEMPORAL ANALYSIS ===")
# Add year-month for temporal analysis
haiti_df['year_month'] = haiti_df['date'].dt.to_period('M')
haiti_df['year'] = haiti_df['date'].dt.year

# Yearly price trends
yearly_stats = haiti_df.groupby('year')['mp_price'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)
print("Yearly price trends:")
print(yearly_stats)

# Price volatility analysis
print(f"\n=== PRICE VOLATILITY ===")
volatility = haiti_df.groupby(['cm_name', 'mkt_name'])['mp_price'].std().reset_index()
volatility = volatility.sort_values('mp_price', ascending=False)
print("Most volatile commodity-market combinations:")
print(volatility.head(10))

In [ ]:
# Data Structure Exploration
print("=== DATASET OVERVIEW ===")
print(f"Shape: {haiti_df.shape}")
print(f"\nColumns: {list(haiti_df.columns)}")
print(f"\nData types:")
print(haiti_df.dtypes)
print(f"\nMemory usage: {haiti_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Convert date column to datetime if it's not already
if haiti_df['date'].dtype == 'object':
    haiti_df['date'] = pd.to_datetime(haiti_df['date'])

print(f"\n=== DATA QUALITY CHECK ===")
print(f"Missing values per column:")
print(haiti_df.isnull().sum())
print(f"\nDuplicate rows: {haiti_df.duplicated().sum()}")

print(f"\n=== UNIQUE VALUES ===")
print(f"Markets: {haiti_df['mkt_name'].nunique()} - {sorted(haiti_df['mkt_name'].unique())}")
print(f"Commodities: {haiti_df['cm_name'].nunique()} - {sorted(haiti_df['cm_name'].unique())}")
print(f"Price types: {haiti_df['pt_name'].nunique()} - {sorted(haiti_df['pt_name'].unique())}")
print(f"Units: {haiti_df['um_name'].nunique()} - {sorted(haiti_df['um_name'].unique())}")
print(f"Currency: {haiti_df['cur_name'].nunique()} - {sorted(haiti_df['cur_name'].unique())}")

print(f"\n=== TIME RANGE ===")
print(f"Date range: {haiti_df['date'].min()} to {haiti_df['date'].max()}")
print(f"Total time span: {(haiti_df['date'].max() - haiti_df['date'].min()).days} days")
print(f"Data frequency: {len(haiti_df['date'].unique())} unique dates")

In [ ]:
# Load WFP Food Price Data for Haiti
# We'll use the WFP Food Prices dataset from Humanitarian Data Exchange

print("Loading WFP food price data for Haiti...")

try:
    # Try to load from WFP/HDX data source
    url = "https://data.humdata.org/dataset/4fdcd4dc-5c2f-4634-b3ca-a08badf7f684/resource/12d7c8e3-eff9-4db0-93b7-726825c4fe9a/download/wfpvam_foodprices.csv"
    
    # Load the data
    df = pd.read_csv(url)
    
    # Filter for Haiti data
    haiti_df = df[df['adm0_name'].str.contains('Haiti', case=False, na=False)].copy()
    
    print(f"Successfully loaded data!")
    print(f"Total global records: {len(df):,}")
    print(f"Haiti records: {len(haiti_df):,}")
    
except Exception as e:
    print(f"Could not load from primary source: {e}")
    print("Creating sample dataset for demonstration...")
    
    # Create sample data for demonstration
    dates = pd.date_range('2020-01-01', '2024-12-31', freq='W')
    markets = ['Port-au-Prince', 'Cap-Haitien', 'Les Cayes', 'Gonaives', 'Jacmel']
    commodities = ['Rice', 'Maize', 'Beans', 'Oil (vegetable)', 'Sugar']
    
    sample_data = []
    np.random.seed(42)
    
    for date in dates:
        for market in markets:
            for commodity in commodities:
                base_price = {'Rice': 50, 'Maize': 30, 'Beans': 80, 'Oil (vegetable)': 120, 'Sugar': 60}[commodity]
                trend = 1 + (date.year - 2020) * 0.15  # Inflation trend
                seasonal = 1 + 0.2 * np.sin(2 * np.pi * date.dayofyear / 365)
                noise = np.random.normal(1, 0.15)
                
                price = max(base_price * trend * seasonal * noise, base_price * 0.5)
                
                sample_data.append({
                    'date': date,
                    'adm0_name': 'Haiti',
                    'adm1_name': market.split('-')[0] if '-' in market else market,
                    'mkt_name': market,
                    'cm_name': commodity,
                    'pt_name': 'Retail',
                    'um_name': 'KG',
                    'mp_price': round(price, 2),
                    'cur_name': 'HTG'
                })
    
    haiti_df = pd.DataFrame(sample_data)
    print(f"Sample dataset created with {len(haiti_df):,} records")

print(f"\nDataset shape: {haiti_df.shape}")
print(f"Date range: {haiti_df['date'].min()} to {haiti_df['date'].max()}")

In [3]:

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',100)


import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from ipywidgets import interact

import itertools, math, time

In [4]:

df = pd.read_csv('wfp_food_prices_hti_2025.csv')

In [5]:
df

,date,admin1,admin2,market,market_id,latitude,longitude,category,commodity,commodity_id,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#loc+market+code,#geo+lat,#geo+lon,#item+type,#item+name,#item+code,#item+unit,#item+price+flag,#item+price+type,#currency+code,#value,#value+usd
1,2005-01-15,Artibonite,Gonaives,Gonaives,103,19.45,-72.68,cereals and tubers,Maize meal (local),471,Marmite,actual,Retail,HTG,60,1.41
2,2005-01-15,Artibonite,Gonaives,Gonaives,103,19.45,-72.68,cereals and tubers,Rice (tchako),57,Marmite,actual,Retail,HTG,94,2.21
3,2005-01-15,Artibonite,Gonaives,Gonaives,103,19.45,-72.68,cereals and tubers,Wheat flour (imported),339,Marmite,actual,Retail,HTG,60,1.41
4,2005-01-15,Centre,Hinche,Hinche,104,19.15,-72.02,cereals and tubers,Maize meal (imported),574,Pound,actual,Retail,HTG,13.16,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15408,2025-07-15,Grande'Anse,Jeremie,Marche de Leon,6023,18.55,-74.11,vegetables and fruits,Mangoes,411,Dozen,actual,Retail,HTG,100,0.76
15409,2025-07-15,Grande'Anse,Jeremie,Marche de Leon,6023,18.55,-74.11,vegetables and fruits,Okra,941,10 pcs,actual,Retail,HTG,50,0.38
15410,2025-07-15,Grande'Anse,Jeremie,Marche de Leon,6023,18.55,-74.11,vegetables and fruits,Oranges,360,Dozen,actual,Retail,HTG,100,0.76
15411,2025-07-15,Grande'Anse,Jeremie,Marche de Leon,6023,18.55,-74.11,vegetables and fruits,Pineapples,524,3 pcs,actual,Retail,HTG,250,1.9
